In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from scipy.ndimage import gaussian_filter1d

from scipy import stats
import statsmodels.formula.api as smf

# For CSD analysis
import Python3_icsd as icsd 
import scipy.signal as sg
import quantities as pq

sns.set_context('poster')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# import Python3_OpenOE_AC_map_functions_v1_08_30s as oem
import mz_LFP_functions as mz_LFP

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    # optional, makes markers bigger, too, axes.linewidth doesn't seem to work
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
    'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
    #'axes.color_cycle':pal, # image.cmap - rewritesd the default colormap
    'axes.linewidth':2.5, 'lines.linewidth': 2.5,
    'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black','axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

#set_pub_bargraphs()
set_pub_plots()

In [ ]:
pal=sns.blend_palette(['black','royalblue'],2)
sns.palplot(pal)
sns.set_palette(pal)

---

# Load some necessary variables

In [4]:
insert_depth = 1000  #change this as appropriate

sp_bw_ch = 20/2
samples_tr = 7350 #this is based on the shortest #samples in a trial
sr = 2500
n_chan = 384
rec_length = 3.0 #how long is the arduino triggered?

---

# First, load in the .npy arrays and CC_ls
These were creaded and saved using the "1_saving_LFP_arrays" jupyter notebook

In [5]:
all_pre_arr = np.load(r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\pre_all.npy")
all_post_arr = np.load(r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\post_all.npy")
all_novel_arr = np.load(r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\novel_all.npy")

In [ ]:
pkl_file = r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\pre_et_ls"

open_file = open(pkl_file, "rb")
et_ls_pre = pickle.load(open_file)
open_file.close()

len(et_ls_pre)

In [ ]:
pkl_file = r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\post_et_ls"

open_file = open(pkl_file, "rb")
et_ls_post = pickle.load(open_file)
open_file.close()

len(et_ls_post)

In [ ]:
pkl_file = r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\novel_et_ls"

open_file = open(pkl_file, "rb")
et_ls_novel = pickle.load(open_file)
open_file.close()

len(et_ls_novel)

---

# Average CSD across all mice in each group

In [9]:
def regroup(all_array, et_ls, show=1):
    sham = []
    nmda = []
    for i in range(all_array.shape[0]):
        nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
        if et_ls[i] in nmda_ls:
            nmda.append(all_array[i])
        else:
            sham.append(all_array[i])

    csd_sham_arr = np.array(sham)
    csd_nmda_arr = np.array(nmda)
    
    if show == 1:
        print(csd_sham_arr.shape)
        print(csd_nmda_arr.shape)
    
    return csd_sham_arr, csd_nmda_arr

In [10]:
sham_pre, nmda_pre = regroup(all_pre_arr, et_ls_pre, show=0)
sham_post, nmda_post = regroup(all_post_arr, et_ls_post, show=0)
sham_novel, nmda_novel = regroup(all_novel_arr, et_ls_novel, show=0)

In [ ]:
strain = 'sham' #input('What type of mouse ("sham", "nmda"): ')
situation = input('What reward situation ("pre", "post", "novel"): ')

if strain == 'sham':
    if situation == 'pre':
        working_data = sham_pre
        data_id = 'Pre training'
    elif situation == 'post':
        working_data = sham_post
        data_id = 'Post training'
    elif situation == 'novel':
        working_data = sham_novel
        data_id = 'Novel'
    else:
        raise Exception('Input is not one of the options')
if strain == 'nmda':
    if situation == 'pre':
        working_data = nmda_pre
        data_id = 'Pre training'
    elif situation == 'post':
        working_data = nmda_post
        data_id = 'Post training'
    elif situation == 'novel':
        working_data = nmda_novel
        data_id = 'Novel'
    else:
        raise Exception('Input is not one of the options')

In [ ]:
# this is a list of the channel x-position along the probe: ch0 at the tip, ch384 at the base
ch_y_pos_ls = [43,11,59,27]
ch_y_pos = np.array(ch_y_pos_ls*int(384/4)) #this repeats the above ls 384/4 times

all_mice_csd = []
for mouse in range(len(working_data)):
    
    single_mouse_lfp = working_data[mouse]
    
    col_ls = []
    for idx, val in enumerate(ch_y_pos):
        if (val == 43) | (val == 59): # all x-values are (43, 11, 59, 27), max and min y-values are (20, 3840)
            col_ls.append(single_mouse_lfp[idx])
    col_arr = np.array(col_ls)
    
    f,ax=plt.subplots(1,2,figsize=(15,8), constrained_layout=True)
    ch_num = col_arr.shape[0]
    csd = mz_LFP.df_CSD_analysis(data=col_arr, axes=ax, Channel_Number=ch_num, up_samp=2, is_plot=0)
    plt.show()
    
    foo_csd = np.array(csd)
    all_mice_csd.append(foo_csd)
    

In [46]:
all_mice_csd = np.array(all_mice_csd)
all_mice_mean = np.mean(all_mice_csd, axis=0)

up_samp = 2
title_str = 'All {0} - '.format(strain) + data_id

In [ ]:
plt.figure(figsize=(6,8))

im = plt.imshow(all_mice_mean, origin='lower', 
                vmin=-5000, vmax=5000,
                cmap='jet', interpolation='nearest',aspect='auto')


plt.title(title_str)
plt.ylabel('Channel #')
plt.xlabel('Time (s)')
plt.xticks([0, 1*2500, 2*2500], labels=[0,1,2])

cb = plt.colorbar(im, aspect=50, shrink=0.8)
cb.formatter.set_powerlimits((0, 0))
cb.update_ticks()
cb.set_label('CSD (%s)' % csd.dimensionality.string)

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\LFP\CSD\novel_dko_CSD.pdf", transparent=True)

plt.show()

In [ ]:
plt.figure(figsize=(6,4))

im = plt.imshow(all_mice_mean, origin='lower', 
                vmin=-2000, vmax=2000,
                cmap='jet', interpolation='nearest',aspect='auto')


plt.title(title_str)
plt.ylabel('Channel #')
plt.xlabel('Time (s)')
plt.ylim([0,115])
plt.xlim([0, 2.0*2500])
plt.xticks([0, 0.5*2500, 1*2500, 1.5*2500, 2.0*2500], labels=[0.0,0.5,1,1.5,2.0])

cb = plt.colorbar(im, aspect=20)
cb.formatter.set_powerlimits((0, 0))
cb.update_ticks()
cb.set_label('CSD (%s)' % csd.dimensionality.string)

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\csd\novel_sham_v1.pdf", transparent=True)

plt.show()

---

---

---

# Plot the CSD for each mouse to see which are best

In [ ]:
situation = input('What reward situation ("pre", "post", "novel"): ')

if situation == 'pre':
    working_data = all_pre_arr
    et_ls = et_ls_pre
    data_id = 'Pre training'
elif situation == 'post':
    working_data = all_post_arr
    et_ls = et_ls_post
    data_id = 'Post training'
elif situation == 'novel':
    working_data = all_novel_arr
    et_ls = et_ls_novel
    data_id = 'Novel'
else:
    raise Exception('Input is not one of the options')
    
print('\nthis data is {0}'.format(data_id))

In [ ]:
for mouse in range(len(working_data)):
    single_mouse_lfp = working_data[mouse]
    
    ch_y_pos_ls = [43,11,59,27]
    ch_y_pos = np.array(ch_y_pos_ls*int(384/4)) #this repeats the above ls 384/4 times

    col_ls = []
    for idx, val in enumerate(ch_y_pos):
        if (val == 43) | (val == 59): # all x-values are (43, 11, 59, 27), max and min y-values are (20, 3840)
    #     if (val == 43):
            col_ls.append(single_mouse_lfp[idx])

    col_arr = np.array(col_ls)
    
    f,ax=plt.subplots(1,2,figsize=(15,8), constrained_layout=True)
    plt.suptitle('Mouse #{0}'.format(mouse))

    ch_num = col_arr.shape[0]

    csd = mz_LFP.df_CSD_analysis(data=col_arr, axes=ax, Channel_Number=ch_num, up_samp=2, is_plot=1)
    plt.show()

# Select single mouse
This requires a _`user input`_ for choosing the scenario to look at!

In [ ]:
mouse_num = int(input('What mouse number (0-{0}): '.format(len(working_data)-1)))

single_mouse_lfp = working_data[mouse_num]

et = et_ls[mouse_num]
print('\nthis mouse is from {0}'.format(et))

nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
if et in nmda_ls:
    mouse_group = 'nmda'
    print('this mouse is a nmda')
else:
    print('this mouse is a sham')
    mouse_group = 'sham'

# Select single column of channels

In [ ]:
# this is a list of the channel x-position along the probe: ch0 at the tip, ch384 at the base
ch_y_pos_ls = [43,11,59,27]
ch_y_pos = np.array(ch_y_pos_ls*int(384/4)) #this repeats the above ls 384/4 times

col_ls = []
for idx, val in enumerate(ch_y_pos):
    if (val == 43) | (val == 59): # all x-values are (43, 11, 59, 27), max and min y-values are (20, 3840)
#     if (val == 43):
        col_ls.append(single_mouse_lfp[idx])
        
col_arr = np.array(col_ls)
col_arr.shape #this is ch x samples for one "column" of recording sites

# CSD Plots of the single mouse
The updated CSD function is on my "mz_LFP_functions.py" file

In [ ]:
plt_title = str('{0} - {1} ({2})'.format(data_id,et,mouse_group))

f,ax=plt.subplots(1,2,figsize=(15,8), constrained_layout=True)
plt.suptitle(plt_title)

ch_num = col_arr.shape[0]

csd = mz_LFP.df_CSD_analysis(data=col_arr, axes=ax, Channel_Number=ch_num, up_samp=2, is_plot=1)

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\csd\novel_dko_1mouse.pdf", transparent=True)

plt.show()

In [ ]:
f,ax=plt.subplots(1,1,figsize=(8,8), constrained_layout=True)

im = plt.imshow(csd, origin='lower', 
                vmin=-5000, vmax=5000, 
                interpolation='nearest', aspect=20, cmap='jet')
cb = plt.colorbar(im, ax=ax, aspect=20, shrink=0.5)
cb.formatter.set_powerlimits((0, 0))
cb.update_ticks()
cb.set_label('CSD (%s)' % csd.dimensionality.string)

ax.set_xlim([0.5*2500, 2.0*2500])
ax.set_ylim([0, 115])
ax.set_yticks([0,50,100])

ax.set_xticks([0.5*2500, 1.25*2500, 2.0*2500])
ax.set_xticklabels([0.5,1.25,2.0])

ax.set_title('V1 channels')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Channel #')

# Change the end of this next line with a new file name!!
# plt.savefig(r"U:\Data_Analysis\zimmer94\01_NMDA_lesions_project\figures_plots\csd\pre_nmda_1mouse_v1.pdf", transparent=True)

plt.show()